In [8]:
import pandas as pd
import numpy as np
import subprocess
from sklearn.manifold import TSNE
from Bio.Cluster import kcluster
import matplotlib.pyplot as plt
from collections import Counter
import glob
import os

## Prepare for analysis

In [9]:
input="TEVp-240412"
input_dataframe=pd.read_csv(f"output/{input}/opt_binders/all.csv")
input_dataframe

,score,plddt,i_ptm,i_pae,i_con,rmsd,model_path,input_pdb,seq
0,0.9639759918683671,0.909887,0.822820,6.196492,2.131226,0.224012,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
1,0.8915366613239336,0.921636,0.832813,5.995022,2.087187,0.260504,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
2,1.019471769526556,0.914033,0.818304,6.036314,2.179966,0.270786,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
3,0.9653374543706308,0.927119,0.849331,5.538220,2.017674,0.360236,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
4,0.9258158285396304,0.919711,0.816974,6.128592,2.135939,0.365271,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
...,...,...,...,...,...,...,...,...,...
28736,0.8201270338496246,0.917276,0.796402,6.600805,2.145624,8.850104,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
28737,0.8199589543393196,0.915942,0.777484,6.749646,2.269633,9.059466,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
28738,0.8079625733564109,0.917571,0.799550,6.426072,2.152792,9.274283,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
28739,0.8193790435209228,0.866297,0.596412,10.632848,2.779262,10.107380,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...


In [10]:
# Functions
# add scaffold name column
def add_scaffold_name_column(filtered, prefix):
    filtered["scaffold_name"] = ""
    
    for index, row in filtered.iterrows():
        path = row["model_path"]
        prefix = prefix
        file_name = path.split("/")[-1]
        parts = file_name.split(prefix)[-1].split("_")
        
        if len(parts) >= 5:
            result = f"{parts[0]}_{parts[1]}_{parts[2]}"
        else:
            result = parts[0].split(".")[0]
        
        filtered.at[index, "scaffold_name"] = result
    
    return filtered

def repeat_rows_by_column_value(df, column_name, number):
    unique_values = df[column_name].unique()
    repeated_rows = []

    for value in unique_values:
        subset = df[df[column_name] == value]
        num_repeats = min(number, subset.shape[0])
        repeated_rows.extend([subset.iloc[i, :] for i in range(num_repeats)])

    repeated_df = pd.DataFrame(repeated_rows)
    return repeated_df

#best_binders=add_scaffold_name_column(best_binders, input+"_")

## Filter dataframe

In [11]:
filtered = input_dataframe[(input_dataframe["rmsd"]<3)&(input_dataframe["plddt"]>0.9)]
#[(input_dataframe["plddt"]>0.7)&(input_dataframe["i_pae"]<8)&(input_dataframe["rmsd"]<3)]

filtered = filtered.sort_values(by='plddt', ascending=False).drop_duplicates("model_path")
filtered

,score,plddt,i_ptm,i_pae,i_con,rmsd,model_path,input_pdb,seq
20975,0.7463607734532854,0.964927,0.911258,4.670973,1.541995,0.356846,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
20986,0.7730128422801477,0.963836,0.904660,4.788784,1.562745,0.486205,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
20959,0.7386843975410512,0.963548,0.903179,4.604320,1.560907,0.282045,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
20988,0.8255736632146796,0.962725,0.900389,4.771775,1.593132,0.604831,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
20943,0.7331290756138055,0.961995,0.894314,4.845370,1.604951,0.180483,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
...,...,...,...,...,...,...,...,...,...
14846,0.7961095624645214,0.900054,0.882009,5.419133,1.860786,0.256984,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
16575,0.9059998060156176,0.900050,0.770064,7.153614,2.352355,0.723585,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
9568,0.780530557316139,0.900050,0.799052,6.610489,2.230887,0.516207,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/af2_best_docks/...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...
14332,0.8160365451266636,0.900043,0.867058,5.662879,1.955603,0.768974,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...


In [12]:
filtered=add_scaffold_name_column(filtered, input+"_")
filtered

,score,plddt,i_ptm,i_pae,i_con,rmsd,model_path,input_pdb,seq,scaffold_name
20975,0.7463607734532854,0.964927,0.911258,4.670973,1.541995,0.356846,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20986,0.7730128422801477,0.963836,0.904660,4.788784,1.562745,0.486205,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20959,0.7386843975410512,0.963548,0.903179,4.604320,1.560907,0.282045,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20988,0.8255736632146796,0.962725,0.900389,4.771775,1.593132,0.604831,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20943,0.7331290756138055,0.961995,0.894314,4.845370,1.604951,0.180483,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
...,...,...,...,...,...,...,...,...,...,...
14846,0.7961095624645214,0.900054,0.882009,5.419133,1.860786,0.256984,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,60_3Hs_20
16575,0.9059998060156176,0.900050,0.770064,7.153614,2.352355,0.723585,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,60_3Hs_4
9568,0.780530557316139,0.900050,0.799052,6.610489,2.230887,0.516207,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/af2_best_docks/...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,27_3H_4
14332,0.8160365451266636,0.900043,0.867058,5.662879,1.955603,0.768974,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,60_3Hs_20


In [13]:
# Calculate statistics on scaffolds
scaffold_counts = filtered["scaffold_name"].value_counts()
total_unique_scaffolds = len(scaffold_counts)
total_scaffold_instances = scaffold_counts.sum()

print("Total unique scaffolds:", total_unique_scaffolds)
print("Total scaffold instances:", total_scaffold_instances)
print("\nScaffold counts:")
print(scaffold_counts)

Total unique scaffolds: 47
Total scaffold instances: 18188

Scaffold counts:
scaffold_name
60_3Hs_29    1586
27_3H_18     1467
lcb3_7_9     1233
lcb3_2_3      832
27_3H_24      786
27_3H_14      769
27_3H_16      739
27_3H_25      717
lcb3_16_9     713
2H3E_18_1     692
27_3H_5       630
27_3H_3       553
lcb3_27_8     471
lcb3_30_5     458
60_3Hs_20     446
27_3H_8       427
lcb3_9_7      409
27_3H_20      381
27_3H_10      371
lcb3_21_5     319
lcb3_14_7     308
lcb3_14_4     291
lcb3_8_4      278
lcb3_28_2     264
51_3H_10      260
lcb3_10_4     240
lcb3_17_9     236
lcb3_15_7     206
60_3Hs_15     195
60_3Hs_4      186
27_3H_15      158
lcb3_22_9     144
lcb3_25_6     140
lcb3_27_9     137
27_3H_4       133
60_3Hs_30     131
51_3H_12      125
51_3H_15      110
lcb3_16_1     105
51_3H_1       102
27_3H_19       85
60_3Hs_5       80
lcb3_21_7      74
lcb3_6_6       68
51_3H_13       62
60_3Hs_7       48
lcb3_7_8       23
Name: count, dtype: int64


In [ ]:
### Check different scaffolds
# filtered=repeat_rows_by_column_value(filtered, "scaffold_name", 1)
# folder=f"/home/tsatler/RFdif/ClusterProteinDesign/scripts/binder_design/output/{input}/opt_binders/test"
# os.makedirs(folder, exist_ok=True)
# for path in filtered["model_path"]:
#     !cp $path $folder

In [ ]:
### Filter by good scaffolds
# good_scaffolds = ["2_","55_","61_","54_","24_","13_","11_"]
# filtered = filtered[~filtered['scaffold_name'].isin(good_scaffolds)]
# filtered

In [14]:
### Redundant scaffolds
designs_per_scaffold = 200

filtered=repeat_rows_by_column_value(filtered, "scaffold_name", designs_per_scaffold)
filtered

,score,plddt,i_ptm,i_pae,i_con,rmsd,model_path,input_pdb,seq,scaffold_name
20975,0.7463607734532854,0.964927,0.911258,4.670973,1.541995,0.356846,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20986,0.7730128422801477,0.963836,0.904660,4.788784,1.562745,0.486205,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20959,0.7386843975410512,0.963548,0.903179,4.604320,1.560907,0.282045,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20988,0.8255736632146796,0.962725,0.900389,4.771775,1.593132,0.604831,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
20943,0.7331290756138055,0.961995,0.894314,4.845370,1.604951,0.180483,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,lcb3_16_9
...,...,...,...,...,...,...,...,...,...,...
13473,0.7752128782122649,0.902551,0.853292,5.601793,2.054863,0.755660,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,51_3H_13
13462,0.9519659455181161,0.902434,0.853703,5.587943,1.998595,0.487871,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,51_3H_13
12579,0.8363335357373569,0.902131,0.835035,5.844373,2.071354,1.530415,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,51_3H_13
13036,0.8492124031742327,0.901524,0.854810,5.802446,2.002744,0.398337,output/TEVp-240412/opt_binders/binders/TEVp-24...,output/TEVp-240412/opt_binders/binders/TEVp-24...,PRDYNPISSTICHLTNESDGHTTSLYGIGFGPFIITNKHLFRRNNG...,51_3H_13


## Cluster sequences

In [9]:
import numpy as np
filtered["seq_split"] = filtered["seq"].apply(lambda x: x.split("/")[-1])
seqs=filtered["seq_split"].to_list()

num_clusters=50

seqs=filtered["seq_split"].to_list()
#matrix = np.asarray([np.frombuffer(seq.encode(), dtype=np.uint8) for seq in seqs])
max_length = max(len(seq) for seq in seqs)
padded_seqs = [seq.ljust(max_length, 'N') for seq in seqs]
matrix = np.asarray([np.frombuffer(seq.encode(), dtype=np.uint8) for seq in padded_seqs])
clusterid, error, nfound = kcluster(matrix, nclusters=num_clusters)

# Apply t-SNE to the matrix to reduce the dimensionality and visualize the sequences.
tsne = TSNE(n_components=2, random_state=42)
embedded_matrix = tsne.fit_transform(matrix)

# Create a scatter plot of the embedded points and label them with cluster IDs.
plt.figure(figsize=(10, 6))
for cluster in range(num_clusters):
    cluster_points = embedded_matrix[clusterid == cluster]
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f"Cluster {cluster}")

plt.title(f"t-SNE Visualization of {input} best protein sequences")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend()
#plt.savefig(f"output/{input}/filtered_sequences/tsne_binders.png")
plt.show()


# Print the number of sequences in each cluster.
cluster_counts = Counter(clusterid)
sorted_cluster_counts = dict(sorted(cluster_counts.items()))
for cluster, count in sorted_cluster_counts.items():
    print(f"Cluster {cluster}: {count} sequences")

# Add cluster id to dataframe
filtered["clusterid"]=clusterid
#filtered.to_csv(f"output/{input}/filtered_sequences/2_filtered_binders_clus.csv", index=False)

# Calculate average cluster metrics
average_metrics_by_cluster = filtered.groupby('clusterid').mean()
#average_metrics_by_cluster.to_csv(f"output/{input}/filtered_sequences/2_cluster_average.csv", index=False)
average_metrics_by_cluster

NameError: name 'np' is not defined

## Prepare metrics command

In [15]:
os.makedirs(f"output/{input}/opt_binders", exist_ok=True)

save_path = f"output/{input}/opt_binders/filtered_binders.csv" # Save filtered
metric_path = f"output/{input}/opt_binders/metrics.csv" # Save filtered with metrics

# Make filtered dataframe or append new sequences to the old one
if os.path.exists(save_path):
    print("reading existant dataframe...")
    existing_dataframe = pd.read_csv(save_path)
    filtered_new = filtered[~filtered["model_path"].isin(existing_dataframe["model_path"])]
    print(f"existing dataframe of len: {len(existing_dataframe)}, new filtered: {len(filtered_new)}")
    existing_dataframe = pd.concat([existing_dataframe, filtered_new], ignore_index=True)
    print(f"final length: {len(existing_dataframe)}")
    existing_dataframe = existing_dataframe.sort_values(by='plddt', ascending=False)
    #drop duplicates
    existing_dataframe.to_csv(save_path, index=False)
    existing_dataframe.to_csv(metric_path, index=False)

else:
    filtered.to_csv(save_path, index=False)
    filtered.to_csv(metric_path, index=False)
    existing_dataframe=filtered

## Prepare input files for analysis script

In [16]:
save_directory = f"output/{input}/opt_binders/analysis_input"

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

batch_size = 1000

# Split the model_paths into batches
model_paths = existing_dataframe["model_path"]
batches = [model_paths[i:i + batch_size] for i in range(0, len(model_paths), batch_size)]

# Save each batch as a separate TXT file
for i, batch in enumerate(batches):
    save_path = os.path.join(save_directory, "model_paths_" + str(i) + ".txt")
    with open(save_path, "w") as file:
        file.write("\n".join(batch))

## Run analysis script

In [17]:
input_files=glob.glob(f"{save_directory}/*txt")
array_limit=300//len(input_files)
target_chain="A"
binder_chain="B"
xml_file="helper_scripts/metrics_calc.xml"

commands=[]

for input_file in input_files:
    with open(input_file, "r") as file:
        lines = file.readlines()
    array_number = len(lines)-1

    bash_arguments=f"--output=/dev/null --array=0-{array_number}%{array_limit}"
    script_arguments=f"{input_file} {target_chain} {binder_chain} {metric_path} {xml_file}"

    command = f"sbatch {bash_arguments} helper_scripts/binder_analysis.sh {script_arguments}"
    print(command)
    commands.append(command)

print(f"This will run {len(commands)} array scripts")

sbatch --output=/dev/null --array=0-999%37 helper_scripts/binder_analysis.sh output/TEVp-240412/opt_binders/analysis_input/model_paths_2.txt A B output/TEVp-240412/opt_binders/metrics.csv helper_scripts/metrics_calc.xml
sbatch --output=/dev/null --array=0-999%37 helper_scripts/binder_analysis.sh output/TEVp-240412/opt_binders/analysis_input/model_paths_0.txt A B output/TEVp-240412/opt_binders/metrics.csv helper_scripts/metrics_calc.xml
sbatch --output=/dev/null --array=0-999%37 helper_scripts/binder_analysis.sh output/TEVp-240412/opt_binders/analysis_input/model_paths_3.txt A B output/TEVp-240412/opt_binders/metrics.csv helper_scripts/metrics_calc.xml
sbatch --output=/dev/null --array=0-999%37 helper_scripts/binder_analysis.sh output/TEVp-240412/opt_binders/analysis_input/model_paths_5.txt A B output/TEVp-240412/opt_binders/metrics.csv helper_scripts/metrics_calc.xml
sbatch --output=/dev/null --array=0-999%37 helper_scripts/binder_analysis.sh output/TEVp-240412/opt_binders/analysis_inp

In [18]:
# Run the array bash script
for command in commands:
    subprocess.run(command, shell=True)

Submitted batch job 395265
Submitted batch job 395266
Submitted batch job 395267
Submitted batch job 395268
Submitted batch job 395269
Submitted batch job 395270
Submitted batch job 395271
Submitted batch job 395272


In [34]:
!squeue --me

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
395265_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395266_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395267_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395268_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395269_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395270_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395271_[37-999%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
395272_[37-705%37]       amd binder_a  lhafner PD       0:00      1 (JobArrayTaskLimit)
          395265_0       amd binder_a  lhafner  R       0:39      1 compute-0-0
          395265_1       amd binder_a  lhafner  R       0:39      1 compute-0-0
          395265_2       amd binder_a  lhafner  R  